In [91]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from nltk.stem import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [3]:
train_dat = pd.read_json("data/train.json")
test_dat = pd.read_json("data/test.json")

In [68]:
# stemmer = SnowballStemmer("english")
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = SnowballStemmer("english")
    def __call__(self, doc):
        return [self.wnl.stem(t) for t in doc.split(" ")]
# Smooth idf, sublinear df, norm
tfIdfTrans = TfidfVectorizer(sublinear_tf=True, tokenizer=LemmaTokenizer())

In [57]:
total_dat = train_dat.drop(['cuisine'],axis=1).append(test_dat)

In [58]:
total_dat["ingredient_str"] = total_dat.ingredients.map(lambda d: ' '.join(d))
total_dat_ing = total_dat.drop(['ingredients'],axis=1)

In [78]:
tfIdfTrans.fit(total_dat_ing.ingredient_str)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<__main__.LemmaTokenizer object at 0x7fcda38a8e10>,
        use_idf=True, vocabulary=None)

In [79]:
vectorIngr = tfIdfTrans.transform(total_dat_ing.ingredient_str[:train_dat.shape[0]]).toarray()

In [81]:
labels = train_dat.cuisine
cv = StratifiedKFold(labels, n_folds=3)

In [93]:
estimator = LogisticRegression()
gausianEstimator = GaussianNB()

In [ ]:
scores = cross_val_score(estimator=gausianEstimator,X=vectorIngr, y=labels, cv=cv, scoring='accuracy', n_jobs=-1)

In [ ]:
scores